<a href="https://colab.research.google.com/github/Motolover/Web_analytics/blob/main/Export_joytoy_to_avito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import lxml
import json
import numpy as np
import html
import time
import os
import sys
import lxml

In [ ]:
from google.colab import files
from lxml import etree

In [ ]:
!pip install gspread_formatting

In [ ]:
import gspread
import oauth2client
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting.dataframe import format_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

In [ ]:
from google.colab import auth

In [ ]:
import yaml

In [ ]:
# Подключение к Google Таблицам
scope = ['https://www.googleapis.com/auth/spreadsheets',
'https://www.googleapis.com/auth/drive']

# Авторизация пользователя с помощью JSON-ключа
credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/My Drive/able-engine-276816-f1ea18570bfa.json', scope)
client = gspread.authorize(credentials)

In [ ]:
gs = gspread.authorize(credentials)
worksheet = gs.open('joytoy_parsing_table').worksheet('Import Matrix 2.0')
print('таблица подключена')

таблица подключена


In [ ]:
rows = worksheet.get_all_values()
joytoy_wh_df = pd.DataFrame.from_records(rows)
joytoy_wh_df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
joytoy_wh_df

,Franchise,SKU (new),SKU (orig),Name,Release,Link,URL,Status,Scale,Price2,Material,Weight,Packing size,Applicable Age,Stock,OMW,Sales,Cost_price,Reasonable_price,Price
0,JoyToy Warhammer 40K,6973130371286,6973130371286,Ultramarines Incursors Sergeant Romulo,01.09.2021,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,-,1:18,119.99,ABS PVC,0.25kg,-,15+,0,0,7,0.00,0,2200
1,JoyToy Warhammer 40K,6973130371287,6973130371286,Ultramarines Incursors Brother Varron,01.09.2021,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,-,1:18,119.99,ABS PVC,0.25kg,-,15+,2,0,5,"1,469.75",1763.703,2200
2,JoyToy Warhammer 40K,6973130371289,6973130371286,Ultramarines Incursors Brother Aurelian,01.09.2021,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,-,1:18,119.99,ABS PVC,0.25kg,-,15+,1,0,6,"1,167.79",1401.342,2200
3,JoyToy Warhammer 40K,6973130371288,6973130371286,Ultramarines Incursors Brother Remus,01.09.2021,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,-,1:18,119.99,ABS PVC,0.25kg,-,15+,1,0,6,"1,167.79",1401.342,2200
4,JoyToy Warhammer 40K,6973130371293,6973130371293,Ultramarines Infiltrators Brother Pullo,01.09.2021,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,-,1:18,119.99,ABS PVC,0.25kg,-,15+,2,0,11,"1,606.74",1928.0895,2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,JoyToy Warhammer 30K,6973130376908,6973130376908,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/A-...,-,1:18,,ABS PVC,,-,15+,0,0,0,0,0,0
393,JoyToy Warhammer 30K,6973130375734,6973130375734,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/B-...,-,1:18,,ABS PVC,,-,15+,0,0,0,0,0,0
394,JoyToy Warhammer 30K,6973130375772,6973130375772,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/C-...,-,1:18,,ABS PVC,,-,15+,0,0,0,0,0,0
395,JoyToy Warhammer 30K,6973130375833,6973130375833,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/D-...,-,1:18,,ABS PVC,,-,15+,0,0,0,0,0,0


In [ ]:
#Создаем ключевые столбцы из парсинга датафрейма
avit_df = joytoy_wh_df[['Franchise', 'SKU (new)', 'Name', 'Link', 'URL', 'Stock', 'Price', 'Scale']]

In [ ]:
avit_df

,Franchise,SKU (new),Name,Link,URL,Stock,Price,Scale
0,JoyToy Warhammer 40K,6973130371286,Ultramarines Incursors Sergeant Romulo,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,0,"2,200",1:18
1,JoyToy Warhammer 40K,6973130371287,Ultramarines Incursors Brother Varron,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,2,"2,200",1:18
2,JoyToy Warhammer 40K,6973130371289,Ultramarines Incursors Brother Aurelian,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,1,"2,200",1:18
3,JoyToy Warhammer 40K,6973130371288,Ultramarines Incursors Brother Remus,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,1,"2,200",1:18
4,JoyToy Warhammer 40K,6973130371293,Ultramarines Infiltrators Brother Pullo,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,2,"2,200",1:18
...,...,...,...,...,...,...,...,...
392,JoyToy Warhammer 30K,6973130376908,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/A-...,0,0,1:18
393,JoyToy Warhammer 30K,6973130375734,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/B-...,0,0,1:18
394,JoyToy Warhammer 30K,6973130375772,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/C-...,0,0,1:18
395,JoyToy Warhammer 30K,6973130375833,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/D-...,0,0,1:18


**Задачем столбцы со статичными значениями**

In [ ]:
Description_string = """В нaличии. Hовый, запечатанный.

{Franchise} {Name}

Oписание модeли:
· Маcштаб пpoдуктa: {Scale}
· Maтериал: АБС-ПВX
· Применимый вoзраcт: 15+
· UРC : {SKU (new)}

Oчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.

❤️ Добавь это объявление к себе в избранное

✅ В профиле есть еще очень много интересного!)

💬 Подписывайся на наши соцсети!

Если интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!

# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, Черные Храмовники, Dark Angels, Темные Ангелы, Grey Knights, Серые Рыцари, Imperial Fists, Имперские Кулаки, Iron Hands, Железные Руки, Necrons, Некроны, Orkcs, Орки, Raven Guards, Гвардия Ворона, Salamanders, Саламандры, Sons of Horus, Сыны Хоруса, T’au Empire, Империя Тау, White Consuls, Белые Консулы, White Scars, Белые Шрамы, Tyranids, Тираниды, Ваха, Коллекционные фигурки, Миниатюры, 1/18, 1/6, League of Legends"""

Description_string

#display(avit_df.style.set_properties(subset=[Description_string'], **{'white-space': 'pre-line'}))

'В нaличии. Hовый, запечатанный.\n\n{Franchise} {Name}\n\nOписание модeли:\n· Маcштаб пpoдуктa: {Scale}\n· Maтериал: АБС-ПВX\n· Применимый вoзраcт: 15+\n· UРC : {SKU (new)}\n\nOчeнь мнoго пoдвижныx суставов, сoответcтвeннo бoльшой выбop пoз для фигурки - к каждой мoдeли идут дополнительные головы, руки, оружие и т.д.\n\n❤️ Добавь это объявление к себе в избранное\n\n✅ В профиле есть еще очень много интересного!)\n\n💬 Подписывайся на наши соцсети!\n\nЕсли интересует что-то из Jоytоy Wаrhаmmеr/Infinity/Dаrk Sоursе, LЕGО, Hot Toys и т.д. пишите - можем привезти под заказ практически любой товар!\n\n# Wаrhаmmеr 40000, Вархаммер 40000, Wаrhаmmеr 40k, JоyТоy, Jоy Тоy, Dеаth Коrрs оf Кriеg, Корпус Смерти Крига, Имперская гвардия, Imреriаl Guаrd, Sрасе Маrinеs, Sрасе Wоlvеs, Ultrаmаrinеs, Ультрамарины, Вlооd Аngеls, Кровавые Ангелы, Аstrа Мilitаrum, Adepta Sororitas, Сестры Битвы, Adeptus Custodes, Кустодес, Adeptus Mechanicus, Механикус, Black Legion, Черный Легион, Black Templars, Черные Хра

In [ ]:
#avit_df['AvitoId'], avit_df['AdStatus'], avit_df['Category'], avit_df['GoodsType'],  avit_df['VehicleType'], avit_df['Address'], avit_df['Condition'], avit_df['AdType'], avit_df['ContactPhone'], avit_df['DateBegin'], avit_df['DateEnd'], avit_df['AllowEmail'], avit_df['ManagerName'], avit_df['ImageNames'], avit_df['Description'], avit_df['Price'] = '', 'Free', 'Коллекционирование', 'Модели', '', 'Москва, метро Улица Скобелевская', 'Новое', 'Продаю своё', '+7 985 182-56-36', '', '', '', '', '', Description_string, ''

In [ ]:
avit_df.loc[:, ['AvitoId', 'AdStatus', 'Category', 'GoodsType', 'VehicleType', 'Address', 'Condition', 'AdType', 'ContactPhone', 'DateBegin', 'DateEnd', 'AllowEmail', 'ManagerName', 'ImageNames', 'Description']] = [
    '', 'Free', 'Коллекционирование', 'Модели', '', 'Москва, метро Улица Скобелевская', 'Новое', 'Продаю своё', '+7 985 182-56-36', '', '', '', '', '', Description_string
]


In [ ]:
avit_df

,Franchise,SKU (new),Name,Link,URL,Stock,Price,Scale,AvitoId,AdStatus,...,Address,Condition,AdType,ContactPhone,DateBegin,DateEnd,AllowEmail,ManagerName,ImageNames,Description
0,JoyToy Warhammer 40K,6973130371286,Ultramarines Incursors Sergeant Romulo,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,0,2200,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
1,JoyToy Warhammer 40K,6973130371287,Ultramarines Incursors Brother Varron,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,2,2200,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
2,JoyToy Warhammer 40K,6973130371289,Ultramarines Incursors Brother Aurelian,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,1,2200,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
3,JoyToy Warhammer 40K,6973130371288,Ultramarines Incursors Brother Remus,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,1,2200,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
4,JoyToy Warhammer 40K,6973130371293,Ultramarines Infiltrators Brother Pullo,https://www.joytoy.com/products/ultramarines-i...,https://img.joytoy.com/u_file/2206/22/photo/01...,2,2200,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,JoyToy Warhammer 30K,6973130376908,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/A-...,0,0,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
393,JoyToy Warhammer 30K,6973130375734,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/B-...,0,0,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
394,JoyToy Warhammer 30K,6973130375772,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/C-...,0,0,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."
395,JoyToy Warhammer 30K,6973130375833,Space Wolves Varagyr Wolf Guard Squad Varagyr ...,https://www.joytoy.com/products/space-wolves-v...,https://img.joytoy.com/u_file/2406/05/photo/D-...,0,0,1:18,,Free,...,"Москва, метро Улица Скобелевская",Новое,Продаю своё,+7 985 182-56-36,,,,,,"В нaличии. Hовый, запечатанный.\n\n{Franchise}..."


In [ ]:
# заменяем в столбце Description заглушку {name} на значение товара из столбца name
def replace_placeholders(row):
  description = row['Description']
  description = description.replace('{Name}', row['Name'])
  description = description.replace('{Franchise}', row['Franchise'])
  description = description.replace('{Scale}', row['Scale'])
  description = description.replace('{SKU (new)}', row['SKU (new)'])
  return description

# Применение функции с использованием .loc[] для присвоения значений
avit_df.loc[:, 'Description'] = avit_df.apply(replace_placeholders, axis=1)

In [ ]:
# Выполняем переименование столбцов под требования шаблона
new_column_name = ['Id', 'Title', 'ImageUrls', 'Stock']
old_column_name = ['SKU (new)', 'Name', 'URL', 'В наличии'] # имя столбца, которое вы хотите изменить
avit_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)

<ipython-input-29-58bbb350dc7d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  avit_df.rename(columns=dict(zip(old_column_name, new_column_name)), inplace=True)


In [ ]:
# Выводим столбцы в порядке шаблона
avit_df_final = avit_df[['Id', 'AvitoId', 'AdStatus', 'Category', 'GoodsType', 'VehicleType', 'Address', 'Title', 'Description', 'Condition', 'Price', 'AdType', 'DateBegin', 'DateEnd', 'AllowEmail', 'ManagerName', 'ContactPhone', 'ImageNames', 'ImageUrls', 'Stock']]
avit_df_final

,Id,AvitoId,AdStatus,Category,GoodsType,VehicleType,Address,Title,Description,Condition,Price,AdType,DateBegin,DateEnd,AllowEmail,ManagerName,ContactPhone,ImageNames,ImageUrls,Stock
0,6973130371286,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Ultramarines Incursors Sergeant Romulo,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,2200,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,0
1,6973130371287,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Ultramarines Incursors Brother Varron,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,2200,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,2
2,6973130371289,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Ultramarines Incursors Brother Aurelian,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,2200,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,1
3,6973130371288,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Ultramarines Incursors Brother Remus,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,2200,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,1
4,6973130371293,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Ultramarines Infiltrators Brother Pullo,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,2200,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2206/22/photo/01...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,6973130376908,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Space Wolves Varagyr Wolf Guard Squad Varagyr ...,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,0,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2406/05/photo/A-...,0
393,6973130375734,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Space Wolves Varagyr Wolf Guard Squad Varagyr ...,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,0,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2406/05/photo/B-...,0
394,6973130375772,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Space Wolves Varagyr Wolf Guard Squad Varagyr ...,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,0,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2406/05/photo/C-...,0
395,6973130375833,,Free,Коллекционирование,Модели,,"Москва, метро Улица Скобелевская",Space Wolves Varagyr Wolf Guard Squad Varagyr ...,"В нaличии. Hовый, запечатанный.\n\nJoyToy Warh...",Новое,0,Продаю своё,,,,,+7 985 182-56-36,,https://img.joytoy.com/u_file/2406/05/photo/D-...,0


###**Import XLSX to Avito**

In [ ]:
#file_path = r"C:\Users\Fibo\Проекты\WH40\avit_df.xlsx"

file_path = "/content/avit_df.xlsx"
avit_df_final.to_excel(file_path, index=False)
# Загружаем файл на локальную машину
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# YML import to VK

In [ ]:
# Шаблон XML
root = etree.Element("yml_catalog", date="2021-04-01 12:20")
shop = etree.SubElement(root, "shop")
name = etree.SubElement(shop, "name")
name.text = "JoyToy - Warhammer 40000, Dark Source, Infinity"
company = etree.SubElement(shop, "company")
company.text = "JoyToy - Warhammer 40000, Dark Source, Infinity"
url = etree.SubElement(shop, "url")
url.text = "https://vk.com/joytoy40k/"

currencies = etree.SubElement(shop, "currencies")
currency = etree.SubElement(currencies, "currency", id="RUB", rate="1")

categories = etree.SubElement(shop, "categories")
category1 = etree.SubElement(categories, "category", id="11")
category1.text = "Warhammer 40000"
category2 = etree.SubElement(categories, "category", id="2")
category2.text = "Dark Source"
category3 = etree.SubElement(categories, "category", id="3")
category3.text = "В наличии"
category4 = etree.SubElement(categories, "category", id="4")
category4.text = "Предзаказ"

offers = etree.SubElement(shop, "offers")

In [ ]:
for _, row in avit_df_final.iterrows():
    offer = etree.SubElement(offers, "offer", id=str(row['Id']), available="true" if row['Stock'] != '0' else "false")
    price = etree.SubElement(offer, "price")
    price.text = str(row['Price'])
    count = etree.SubElement(offer, "count")
    count.text = str(row['Stock'])
    currencyId = etree.SubElement(offer, "currencyId")
    currencyId.text = "RUB"
    name = etree.SubElement(offer, "name")
    name.text = row['Title']
    description = etree.SubElement(offer, "description")
    description.text = 'Title'# для теста ставлю значение
    image_urls = row['ImageUrls'].split(' | ')
    for image_url in image_urls:
        picture = etree.SubElement(offer, "picture")
        picture.text = image_url.strip()
    # Добавление пустого тега <offers> после каждого <offer>
    offers = etree.SubElement(shop, "offers")

tree = etree.ElementTree(root)
tree.write("output.xml", pretty_print=True, xml_declaration=True, encoding="utf-8")

In [ ]:
# Скачать файл на локальный компьютер
files.download("output.xml")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<yml_catalog date="2021-04-01 12:20">
  <shop>
    <name>JoyToy - Warhammer 40000, Dark Source, Infinity</name>
    <company>JoyToy - Warhammer 40000, Dark Source, Infinity</company>
    <url>https://vk.com/joytoy40k/</url>
    <currencies>
      <currency id="RUB" rate="1"/>
    </currencies>
    <categories>
      <category id="11">Warhammer 400001</category>
      <category id="2">Dark Source</category>
      <category id="3">В наличии</category>
      <category id="4">Предзаказ</category>
    </categories>
    <offers>
      <offer id="6973130379572" available="true">
        <price>3500</price>
        <count>0</count>
        <currencyId>RUB</currencyId>
        <name>Sons of Horus MKIV Sergeant with Power Fist</name>
        <description>Sons of Horus MKIV Sergeant with Power Fist</description>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7488.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7489.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7493.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7494.jpg</picture>
        <picture>https://img.joytoy.com/u_file/2402/29/photo/262A7539.jpg</picture>
      </offer>